cd /data/espnet/docker/prebuilt/devel/gpu

# Start

In [1]:
from glob import glob
import os
import sys
from tqdm import tqdm
import numpy as np
import time
import pandas as pd

# 读取数据
data_path = 'm_0.65_0.85'
fileNames = glob(data_path+'/*0.txt')
_idx = np.arange(len(fileNames))
np.random.shuffle(fileNames)
fileNames[:10]


def dump(sr=None):

    if sr is None: # 从配置文件中获取 sr
        with open(data_path+'/输入参数文本.txt') as _f:
            _paras = _f.readlines()
            sr = int(_paras[2].split('\t')[0]) + 1
            print('sr=',sr)

    # --------------------------------------------------------------通用begin
    with open(fileNames[0]) as _f:
        simple_size = sys.getsizeof(_f.read())/1024/1024 # Mb


    filename = fileNames
# --------------------------------------------------------------通用end
    i1 = np.zeros((len(filename), sr))
    i2 = np.zeros((len(filename), sr))
    dop = np.zeros((len(filename), sr))
    M,ALPHA = [], []
    t0 = time.time()
    L = len(filename)
    for idx, i in enumerate(filename):
        baseName = os.path.basename(i) 
        m,alpha = baseName.split(',')
        alpha = '.'.join(alpha.split('.')[:-1])
        m = m.split('=')[-1]
        alpha=alpha.split('=')[-1]
        M.append(m)
        ALPHA.append(alpha)

        i1[idx] = pd.read_table(i, encoding='gb2312',sep='\t',index_col=None).I1.to_numpy().astype(np.float32)
        i2[idx] = pd.read_table(i, encoding='gb2312',sep='\t',index_col=None).I2.to_numpy().astype(np.float32)
        dop[idx] = pd.read_table(i, encoding='gb2312',sep='\t',index_col=None).DOP.to_numpy().astype(np.float32)
        
        t1 = time.time()
        per_current = (idx+1)/L
        cost_time = t1-t0
        Totle_time = cost_time/per_current
#         line_ = 100*(per_current)//2 * '='
        print(f'\r{cost_time:.1f}s |{Totle_time-cost_time:.1f}s ',end='', flush=True)
    
    M, ALPHA = np.array(M).astype('float32'), np.array(ALPHA).astype('float32')
    # --------------------------------------------------------------- 通用begin
    out_path = '0.65_out_shuffle'
    np.save(os.path.join(out_path, f'i1_{0}.npy'),i1)
    np.save(os.path.join(out_path, f'i2_{0}.npy'),i2)
    np.save(os.path.join(out_path, f'dop_{0}.npy'),dop)
    np.savez(os.path.join(out_path, f'M_ALPHA_{0}.npz'),M=M,ALPHA=ALPHA)

    # --------------------------------------------------------------- 通用end
    return out_path, sr

target_path, sr = dump(sr=2048)

In [2]:
target_path, sr = '0.65_out_shuffle', 2048

In [4]:
# # load
# def load_data(path):
#     fileNames = glob(os.path.join(path,'*'))
    
#     name_i1 = [i for i in fileNames if 'i1' in i]
#     name_i2 = [i for i in fileNames if 'i2' in i]
#     name_dop = [i for i in fileNames if 'dop' in i]
#     name_m_alpha = [i for i in fileNames if 'M_ALPHA' in i]
#     print(name_i1,'\n',name_i2,'\n',name_m_alpha)
#     L = len(name_dop) # 3种文件数目相同
#     c = 0
#     while True:
#         i1 = np.load(name_i1[c%L])
#         i2 = np.load(name_i2[c%L])
#         dop = np.load(name_dop[c%L])
#         res = np.load(name_m_alpha[c%L])
#         M, ALPHA = res['M'], res['ALPHA']
#         print(i1.shape, i2.shape, M.shape)
#         c += 1
#         yield i1, i2, dop, M, ALPHA
        

# data = load_data(target_path)
# i1, i2, dop, M, ALPHA = next(data)

['0.65_out_shuffle/i1_0.npy'] 
 ['0.65_out_shuffle/i2_0.npy'] 
 ['0.65_out_shuffle/M_ALPHA_0.npz']
(50500, 2048) (50500, 2048) (50500,)


In [5]:
# i1, i2, dop, M, ALPHA = next(data)
# i1.shape, i2.shape, dop.shape, M.shape, ALPHA.shape


(50500, 2048) (50500, 2048) (50500,)


((50500, 2048), (50500, 2048), (50500, 2048), (50500,), (50500,))

In [6]:
# import matplotlib.pyplot as plt
# IDX = 1000
# plt.figure(figsize=(16,8))
# plt.plot(np.log(i1[IDX]))
# M[IDX], ALPHA[IDX]

(0.822, 50.5)

In [7]:
# def generateData(batch,theta1, theta2,i1, i2, dop,M, ALPHA, sr, L=i1.shape[0]):
#     S = np.arange(L-1)

#     d_the = (sr-1)/180
#     th1, th2 = int(theta1*d_the), int(theta2*d_the)
#     SR = th2-th1
#     while True:
#         np.random.shuffle(S)
#         series = S[:batch]
#         yield SR,\
#         i1[series, th1:th2].astype(np.float32)[...,None], \
#         i2[series, th1:th2].astype(np.float32)[...,None], \
#         dop[series, th1:th2].astype(np.float32)[...,None], \
#         np.array(M)[series].astype(np.float32)[...,None], \
#         np.array(ALPHA)[series].astype(np.float32)[...,None]
        
# theta1, theta2 = 15, 36

# data = generateData(15, theta1,theta2,i1,i2,dop,M,ALPHA,sr)
# SR = next(data)[0]

In [8]:
print(next(data)[0])
print(next(data)[1].shape) # i1
print(next(data)[2].shape) # i2
print(next(data)[3].shape) # dop
print(next(data)[4].shape) # M
print(next(data)[5].shape) # ALPHA

239
(15, 239, 1)
(15, 239, 1)
(15, 239, 1)
(15, 1)
(15, 1)


In [3]:

import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
# tf.test.is_gpu_available()
from tensorflow.keras import Input,Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Concatenate, Conv2D,Conv1D,Convolution1D,\
Flatten, BatchNormalization,Input,Dropout
from tensorflow.keras.layers import LeakyReLU, ReLU, LayerNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adadelta, Adam, SGD
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy, MAE, KLD, MSLE, MSE

In [10]:
# # Model 1
# def model1(SR):
#     input_x = Input(shape=(SR, 1))
#     # x = LayerNormalization()(input_x)
#     x = input_x

#     x = Conv1D(3, 4, strides=2,padding='same')(x)
#     # x = BatchNormalization()(x)
#     x = ReLU()(x)
#     x = Conv1D(3, 3, strides=2,padding='same')(x)

#     x = ReLU()(x)
#     x = Conv1D(3, 3, strides=2,padding='same')(x)

#     x = ReLU()(x)
#     x = Conv1D(3, 3, strides=2,padding='same')(x)

#     x = ReLU()(x)
#     x = Conv1D(3, 3, strides=2,padding='same')(x)

#     x = ReLU()(x)
#     x = Flatten()(x)

#     x = Dense(100,'relu')(x)
#     rm = Dense(1,'sigmoid')(x)
#     alpha = Dense(1,'relu')(x)
#     dop = Dense(1, 'tanh')(x)
#     model_m = Model(input_x, rm)
#     model_alpha = Model(input_x, alpha)
#     model_dop = Model(input_x, dop)
#     return model_m, model_alpha, model_dop

In [11]:
# # Model 2
# def model2(SR):
#     input_x = Input(shape=(SR, 1))
#     # x = LayerNormalization()(input_x)
    
#     # convolution part
#     cx = input_x
#     cx = Conv1D(1, SR//10, strides=2,padding='valid', activation='relu')(cx)
#     cx = Conv1D(1, 3, strides=2,padding='valid', activation='relu')(cx)
#     cx = Conv1D(1, 3, strides=2,padding='valid', activation='relu')(cx)

#     cx = Flatten()(cx)
    
#     # full connection part
#     fx = input_x
#     fx = Flatten()(fx)
#     fx = Dense(SR//20,'relu')(fx)

# #     x = fx

#     x = tf.concat([fx, cx], axis=1)
#     print(x.shape)
#     rm = Dense(1,'sigmoid')(x)
#     alpha = Dense(1, 'relu')(x)
#     dop = Dense(1, 'tanh')(x)
    
#     model_m = Model(input_x, rm)
#     model_alpha = Model(input_x, alpha)
#     model_dop = Model(input_x, dop)
#     return model_m, model_alpha, model_dop


In [12]:
# # Model 3
# def model3(SR):
#     input_x = Input(shape=(SR, 1))
#     # x = LayerNormalization()(input_x)
    
#     # convolution part
#     x = Flatten()(input_x)
    
#     x = Dense(SR*2,'relu')(x)
#     x = Dense(SR*2,'relu')(x)
#     x = Dense(SR*2,'relu')(x)
#     x = Dense(SR*2,'relu')(x)
#     x = Dense(SR,'relu')(x)
#     x = Dense(SR,'relu')(x)

#     rm = Dense(1,'sigmoid')(x)
#     alpha = Dense(1, 'relu')(x)
#     dop = Dense(1, 'sigmoid')(x)
    
#     model_m = Model(input_x, rm)
#     model_alpha = Model(input_x, alpha)
#     model_dop = Model(input_x, dop)
#     return model_m, model_alpha, model_dop

In [13]:
# model_rm, model_alpha, model_dop = model3(SR)

# model = model_alpha
# # model = model_rm
# model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 239, 1)]          0         
_________________________________________________________________
flatten (Flatten)            (None, 239)               0         
_________________________________________________________________
dense (Dense)                (None, 478)               114720    
_________________________________________________________________
dense_1 (Dense)              (None, 478)               228962    
_________________________________________________________________
dense_2 (Dense)              (None, 478)               228962    
_________________________________________________________________
dense_3 (Dense)              (None, 478)               228962    
_________________________________________________________________
dense_4 (Dense)              (None, 239)               1144

In [14]:
# model.compile(loss= MSE,
#               optimizer=Adam(learning_rate=2e-4, beta_1=0.5)
# #               optimizer=SGD()
#              )

In [32]:
# # train with rm 
# batch = 900
# theta1, theta2 = 15, 36
# DATA = load_data(target_path)
# Big_loop = 0
# while Big_loop < 1:

#     i1,i2,dop,M,ALPHA = next(DATA)
#     Big_loop += 1
#     data = generateData(batch,theta1,theta2,i1,i2,dop,M,ALPHA,sr, i1.shape[0])
    
#     # 使用集成度较低的 Model.train_on_batch方法，自行设定batch数据　和　epochs
#     for step in range(5000):
#         train_data = next(data)
#     #     cost = model.train_on_batch(train_data[1], train_data[-2]) # i1
#     #     cost = model.train_on_batch(train_data[2], train_data[-2]) # i2
#         cost = model.train_on_batch(train_data[3], train_data[-2]) # dop

#         if step%500==0:
#         print(f'loss: {cost}')
#     model.save(f'rm_model_{step}.h5')

['0.65_out_shuffle/i1_0.npy'] 
 ['0.65_out_shuffle/i2_0.npy'] 
 ['0.65_out_shuffle/M_ALPHA_0.npz']
(50500, 2048) (50500, 2048) (50500,)
loss: 0.00018266504048369825


In [33]:
# batch=10
# data = generateData(batch,theta1,theta2,i1,i2,dop,M,ALPHA,sr, i1.shape[0])
# train_data = next(data)
# test_dop, test_rm = train_data[3], train_data[-2]
# test_dop.shape, test_rm.shape

((10, 239, 1), (10, 1))

In [34]:
# for i, j in zip(model.predict(test_dop), test_rm):
#     print(i,j)

[0.7758099] [0.774]
[0.81077933] [0.808]
[0.7611233] [0.76]
[0.7962868] [0.796]
[0.8339187] [0.832]
[0.75567824] [0.654]
[0.81595075] [0.816]
[0.68885803] [0.688]
[0.78161675] [0.784]
[0.6986482] [0.698]


In [4]:
def test_simple(theta1, theta2, alpha, rm, noise=False):
    test_i1, test_i2, test_dop = gen(alpha = alpha, rm = rm, N_ta = sr-1) # 
    
    d_the = (sr-1)/180
    th1, th2 = int(theta1*d_the), int(theta2*d_the)
    SR = th2-th1
    test_i1, test_i2, test_dop = test_i1[th1:th2], test_i2[th1:th2], test_dop[th1:th2]
    
    if noise:
        noise = np.random.rand(SR, 1) * 0.05
        noise_dop = (np.random.rand(SR, 1)-0.5) * 0.05
        test_i1, test_i2, = test_i1+noise, test_i2+noise
        test_dop = test_dop + noise_dop
    return test_i1, test_i2, test_dop 

def test_batch():
    RM = [0.772, 0.772, 0.772, 0.772, 0.772, 0.772, 0.772]
    ALPHA=[5.2,  20.2, 50.2, 70.2, 90.2, 100.2,140.2]
    RES = np.zeros((len(RM),len(ALPHA)))
    for ir, rm in enumerate(RM):
        for ia, alpha in enumerate(ALPHA):
            test_dop = test_simple(theta1, theta2,alpha,rm, noise=0)[2]
            pred_rm = model.predict(test_dop[None,])
            
            RES[ir][ia] = pred_rm
    
    mean_rm = np.mean(RES, axis=1)
    std_rm = np.std(RES, axis=1)
    return std_rm, mean_rm

In [5]:
def test_generateData(alpha,rm,im =0.0,N_ta = 2**13-1):
    origin_path = os.getcwd()
    os.chdir(r"C:\Users\beidongjiedeguang\Desktop\测试文件夹")
#     os.chdir(r"C:\Users\Administrator\OneDrive\Mie测试文件夹")
    exeName = "Mie_Calculation.exe"
    txtName = '输入参数文本.txt'

    Ta_min, Ta_max = 0,180
    with open(txtName,'w') as f1:
        f1.write('{}\n{:.3f}\t{:.3f}\n{}\t{:.2f}\t{:.2f}'
                .format(alpha,rm,im,N_ta,Ta_min,Ta_max))

    os.system(exeName)
    dataName = "散射光强角分布.txt"
    # 没想到c写出来的不是utf-8编码的，所以这里encoding='gbk'，后面这个delimiter随便要不要
    data = pd.read_csv(dataName,sep ='\t',engine='python',encoding='gbk',delimiter='\s+')
    os.remove(dataName)
    os.chdir(origin_path)
    return data

def gen(alpha = 900, rm = 1.7, im= 0.0, N_ta = 2**13-1):
    data = test_generateData(alpha,rm,im, N_ta = N_ta)
    theta = data['散射角'].to_numpy()
    i1, i2, DOP = data['I1'].to_numpy(), data['I2'].to_numpy(), data['可视度'].to_numpy()
    return i1[...,None], i2[...,None], DOP[...,None]



In [6]:
from tensorflow.keras.models import load_model

In [24]:
# load model
model = load_model('rm_model_5000.h5') 
theta1, theta2 = 15, 36

In [9]:
theta1, theta2 = 15, 36
test_batch()

(array([39.67944779, 39.67944779, 39.67944779, 39.67944779, 39.67944779,
        39.67944779, 39.67944779]),
 array([65.92524263, 65.92524263, 65.92524263, 65.92524263, 65.92524263,
        65.92524263, 65.92524263]))

In [28]:
test_alpha, test_rm = 39, 0.7
test_i1, test_i2, test_dop = test_simple(theta1, theta2, alpha=test_alpha, rm=test_rm, noise=False)
model.predict(test_dop[None,...])

array([[0.70198685]], dtype=float32)

In [7]:
model = load_model('alpha_model_5000.h5') 
theta1, theta2 = 15, 36

In [23]:
test_alpha, test_rm = 110, 0.8
test_i1, test_i2, test_dop = test_simple(theta1, theta2, alpha=test_alpha, rm=test_rm, noise=False)
model.predict(test_dop[None,...])

array([[113.35775]], dtype=float32)